In [ ]:
# 地质灾害监测异常检测分析

本notebook演示了如何使用专门为地质灾害监测数据开发的异常检测系统。

该系统支持7种监测数据类型：
- 泥水位监测
- 倾角监测  
- 加速度监测
- GNSS位移监测
- 含水率监测
- 雨量监测
- 裂缝监测

## 1. 导入必要的库和模块


In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
from hive_data_loader import HiveDataLoader
from geological_anomaly_detector import GeologicalAnomalyDetector
from main_analysis import GeologicalAnomalyAnalyzer

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("库导入完成")


In [ ]:
## 2. 数据加载与探索

使用HiveDataLoader加载7张监测数据表


In [ ]:
# 初始化数据加载器
loader = HiveDataLoader()

# 查看所有可用的表
print("可用的监测数据表:")
for table_key in loader.get_table_list():
    description = loader.get_table_description(table_key)
    print(f"  {table_key}: {description}")

# 加载所有表的数据
print("\n正在加载所有监测数据...")
all_data = loader.load_all_tables()

# 显示每个表的基本信息
print("\n各表数据概览:")
for table_key, df in all_data.items():
    print(f"\n{table_key}:")
    print(f"  形状: {df.shape}")
    print(f"  列名: {list(df.columns)}")
    print(f"  时间范围: {df['create_time_s'].min()} 至 {df['create_time_s'].max()}")
    print(f"  监测点数量: {df['monitor_point_code'].nunique()}")
    
# 构建特征矩阵
print("\n构建特征矩阵...")
feature_matrix = loader.get_feature_matrix(all_data)
print(f"特征矩阵形状: {feature_matrix.shape}")
print(f"特征列: {[col for col in feature_matrix.columns if col not in ['monitor_point_code', 'create_time_s']]}")

# 显示特征矩阵的前几行
print("\n特征矩阵预览:")
feature_matrix.head()


In [ ]:
## 3. 运行完整的异常检测分析

使用GeologicalAnomalyAnalyzer进行端到端的异常检测分析


In [ ]:
# 创建异常检测分析器
analyzer = GeologicalAnomalyAnalyzer(output_dir="../outputs")

# 运行完整的异常检测分析
print("开始运行完整的异常检测分析...")
analysis_results = analyzer.run_complete_analysis(
    contamination=0.1,        # 异常值比例
    window_size=24           # 时间窗口大小（小时）
)

print("\n分析完成！")

# 显示分析摘要
summary = analyzer.get_analysis_summary()
print("\n=== 分析摘要 ===")
for key, value in summary.items():
    print(f"{key}: {value}")

# 显示异常检测结果
print("\n=== 异常检测结果 ===")
multivariate_results = analysis_results['anomaly_detection']['multivariate_results']
for method, result in multivariate_results['by_method'].items():
    print(f"{method}: 异常数量={result['anomaly_count']}, 异常率={result['anomaly_rate']:.2%}")

# 显示特征重要性
print("\n=== 特征重要性 TOP 5 ===")
feature_importance = analysis_results['anomaly_detection']['feature_importance']
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
for feature, importance in sorted_features[:5]:
    print(f"{feature}: {importance:.3f}")

# 显示建议
print("\n=== 建议 ===")
for rec in multivariate_results['recommendations']:
    print(f"- {rec}")
